<a href="https://colab.research.google.com/github/masolanoo/x-rays/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#pre

In [1]:
from google.colab import drive 
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
! pip install kaggle
! mkdir ~/.kaggle
! cp drive/MyDrive/kaggle/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
# #etiquetas
# ! kaggle datasets download nih-chest-xrays/data -f Data_Entry_2017.zip

# #train_val_list.txt
# ! kaggle datasets download nih-chest-xrays/data -f train_val_list.zip

# #test_list.txt
# ! kaggle datasets download nih-chest-xrays/data -f test_list.zip

404 - Not Found
404 - Not Found
404 - Not Found


In [74]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns 
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot, plot
import plotly as py
init_notebook_mode(connected=True)
import plotly.graph_objs as go


Datos de las imágenes

In [64]:
df = pd.read_csv('Data_Entry_2017.csv')
df

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
112115,00030801_001.png,Mass|Pneumonia,1,30801,39,M,PA,2048,2500,0.168,0.168,NaN
112116,00030802_000.png,No Finding,0,30802,29,M,PA,2048,2500,0.168,0.168,NaN
112117,00030803_000.png,No Finding,0,30803,42,F,PA,2048,2500,0.168,0.168,NaN
112118,00030804_000.png,No Finding,0,30804,30,F,PA,2048,2500,0.168,0.168,NaN


In [65]:
df.shape

(112120, 12)

In [71]:
#encontrar lista de hallazgos/patologías posibles
labels = set() 
for i in pd.unique(df['Finding Labels']):
    labels.update(i.split('|'))

#quitar 'No Finding'
labels.remove('No Finding')

labels = list(labels)

unique_com = len(pd.unique(df['Finding Labels']))
print(f'Así, existen {unique_com} combinaciones de etiquetas únicas \ny {len(labels)} posibles condiciones: {labels}')

#crear nuevas columnas en el dataframe que representen cada una de estas etiquetas
for label in labels:
    df[label] = df['Finding Labels'].map(lambda x: 1 if label in x else 0)


#revisar dataframe
df.loc[:,['Image Index', 'Finding Labels']+labels]

Así, existen 836 combinaciones de etiquetas únicas 
y 14 posibles condiciones: ['Atelectasis', 'Pleural_Thickening', 'Hernia', 'Consolidation', 'Pneumothorax', 'Cardiomegaly', 'Pneumonia', 'Effusion', 'Emphysema', 'Mass', 'Nodule', 'Edema', 'Fibrosis', 'Infiltration']


,Image Index,Finding Labels,Atelectasis,Pleural_Thickening,Hernia,Consolidation,Pneumothorax,Cardiomegaly,Pneumonia,Effusion,Emphysema,Mass,Nodule,Edema,Fibrosis,Infiltration
0,00000001_000.png,Cardiomegaly,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,00000001_001.png,Cardiomegaly|Emphysema,0,0,0,0,0,1,0,0,1,0,0,0,0,0
2,00000001_002.png,Cardiomegaly|Effusion,0,0,0,0,0,1,0,1,0,0,0,0,0,0
3,00000002_000.png,No Finding,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,00000003_000.png,Hernia,0,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112115,00030801_001.png,Mass|Pneumonia,0,0,0,0,0,0,1,0,0,1,0,0,0,0
112116,00030802_000.png,No Finding,0,0,0,0,0,0,0,0,0,0,0,0,0,0
112117,00030803_000.png,No Finding,0,0,0,0,0,0,0,0,0,0,0,0,0,0
112118,00030804_000.png,No Finding,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [82]:
# # data preparation
# df2016 = timesData[timesData.year == 2016].iloc[:7,:]
# pie1 = df2016.num_students
# pie1_list = [float(each.replace(',', '.')) for each in df2016.num_students]  # str(2,4) => str(2.4) = > float(2.4) = 2.4
# labels = df2016.university_name
# figure
def make_piechart(values, labels, name='', title="título", text='text'):
    fig = {
    "data": [
        {
        "values": values,
        "labels": labels,
        #"domain": {"x": [0, .5]},
        "name": name,
        "hoverinfo":"label+percent+name",
        "hole": .3,
        "type": "pie"
        },],
    "layout": {
            "title":title,
            "annotations": [
                { "font": { "size": 20},
                "showarrow": False,
                "text": text,
                    "x": 0.20,
                    "y": 1
                },
            ]
        }
    }
    return fig
la = ["Hernia"]
for label in la:
    values = df[label]
    labs = ['Yes', 'No']
    fig = make_piechart(values, labs)
    iplot(fig)